In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
import datetime


news_df = pd.read_csv('../data/raw_analyst_ratings.csv')
news_df.head()
news_df.info()


In [ ]:
news_df['headline_length'] = news_df['headline'].apply(len)
print(news_df['headline_length'].describe())

# Count articles per publisher
publisher_counts = news_df['publisher'].value_counts()
publisher_counts.plot(kind='bar', title='Articles per Publisher')


In [ ]:

news_df['date'] = pd.to_datetime(news_df['date'])
news_df.set_index('date')['headline'].resample('D').count().plot(figsize=(12,5), title='Articles Published Over Time')


In [ ]:
# Top publishers
top_publishers = news_df['publisher'].value_counts().head(10)

# Unique email domains (if applicable)
if news_df['publisher'].str.contains('@').any():
    news_df['domain'] = news_df['publisher'].str.extract(r'@([\w.]+)')
    print(news_df['domain'].value_counts())


In [ ]:
%pip install scikit-learn

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocess text
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform(news_df['headline'].fillna(""))

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

# Display topics
for index, topic in enumerate(lda.components_):
    print(f"TOP 10 WORDS FOR TOPIC #{index}")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print()
